# Preprocessings

In this notebook we develop and evaluate the various preprocessing both for segmentation and classification which should help make our solution invariant to some data differences and make ML feature more powerfull.

In [ ]:
from notebook_utils import *

import imageio
import skimage.segmentation
import skimage.filters

import scipy.stats

import random

import os
import shutil

%matplotlib inline

## Load input data

In [ ]:
import clb.dataprep.utils
dir_root = r'D:\Fafa\MIT\CellDx\preprocessing'

In [ ]:
path_3d = os.path.join(dir_root, 'TestNewClasses S2 1024 crop_class_pdlcd_8.tif')
data_input_bad_full = imageio.volread(path_3d)
data_input_bad = imageio.volread(path_3d)[::,300:500,150:350,::]
data_input_bad_normalized = clb.dataprep.utils.rescale_to_float(data_input_bad, float_type='float32')

path_3d_labels = os.path.join(dir_root, 'TestNewClasses S2 1024 crop_class_pdlcd_8_segmented.tif')
data_input_bad_labels = imageio.volread(path_3d_labels)[::,300:500,150:350]
show_all(2,3,data_input_bad[0], data_input_bad[0][::,::,0], data_input_bad_labels[0], data_input_bad[0][::,::,1], data_input_bad[0][::,::,2], data_input_bad_labels[0], scale=10)

In [ ]:
path_3d = os.path.join(dir_root, 'TestNewClasses S1 1024 crop_class_pdlcd_9.tif')
data_input = imageio.volread(path_3d)[::,150:350,150:350,::]
data_input_normalized = clb.dataprep.utils.rescale_to_float(data_input, float_type='float32')

path_3d_labels = os.path.join(dir_root, 'TestNewClasses S1 1024 crop_class_pdlcd_9_segmented.tif')
data_input_labels = imageio.volread(path_3d_labels)[::,150:350,150:350]
show_all(2,3,data_input[0], data_input[0][::,::,0], data_input_labels[0], data_input[0][::,::,1], data_input[0][::,::,2], data_input_labels[0], scale=20)

## Clahe

In [ ]:
from skimage import exposure
from skimage.morphology import disk
from skimage.filters.rank import median

def median_colour_filter(image, size):
    image = (image * 255).astype(np.uint8)
    for i in range(3):
        image[::,::,i] = median(image[::,::,i], disk(size))
    return image

def clahe(image, size, median, **kwargs):
    if median is not None:
        image = median_colour_filter(image, median) / 255.0
    data_clahe = exposure.equalize_adapthist(image, size, **kwargs)
    return data_clahe

In [ ]:
from skimage import exposure

sample = data_input_normalized[0]
data_clahe = clahe(sample, 70, median=2, clip_limit=0.015)
print(data_clahe.shape, data_clahe.dtype)
show_all(3,1,
         np.hstack([sample[::,::,0], data_clahe[::,::,0]]), 
         np.hstack([sample[::,::,1], data_clahe[::,::,1]]), 
         np.hstack([sample[::,::,2], data_clahe[::,::,2]]), scale=30);

In [ ]:
sample_bad = data_input_bad_normalized[0]
data_bad_clahe = clahe(sample_bad, 70, 2, clip_limit=0.015)
show_all(3,1,
         np.hstack([sample_bad[::,::,0], data_bad_clahe[::,::,0]]), 
         np.hstack([sample_bad[::,::,1], data_bad_clahe[::,::,1]]), 
         np.hstack([sample_bad[::,::,2], data_bad_clahe[::,::,2]]), scale=30);

### Final version

In [ ]:
import clb.classify.feature_extractor as fe

print (data_input_bad_normalized.shape, data_input_bad_normalized.dtype)
pre_clahe_bad, pre_clahe_labels = fe.preprocess_channel(data_input_bad_normalized[::, ::,::,0], data_input_bad_labels, 'clahe')
print (pre_clahe_bad.shape, pre_clahe_bad.dtype)
show_all(2,1, np.hstack([sample_bad[::,::,0], pre_clahe_bad[0]]), 
                        np.hstack([data_input_bad_labels[0], pre_clahe_labels[0]]), scale=30)

## Edges

In [ ]:
from skimage import exposure
from skimage.morphology import disk
from skimage.filters.rank import median
import skimage.filters
import scipy.stats
import scipy.ndimage

def median_colour_filter(image, size):
    image = image.copy()
    for i in range(3):
        image[::,::,i] = median(image[::,::,i], disk(size))
    return image

def log(image, size, median, **kwargs):
    if median is not None:
        image = median_colour_filter(image, median) / 255.0
    data_clahe = exposure.equalize_adapthist(image, size, **kwargs)
    return data_clahe

In [ ]:
logas2 = scipy.ndimage.gaussian_laplace(sample[::,::,0], 1)
print(logas2.shape)
print(scipy.stats.describe(logas2, None))

logas2b = scipy.ndimage.gaussian_laplace(sample[::,::,0], 1.5)
print(logas2b.shape)
print(scipy.stats.describe(logas2b, None))

show_all(1,1, np.hstack([logas2, logas2b]), scale=20)

In [ ]:
#values = [i / 10 for i in range(1,20, 1)]
#res = [scipy.ndimage.gaussian_laplace(sample[::,::,0], v) for v in values ]
#show_all(len(res) // 4, 4, *res, scale=40, titles = list(map(str,values)))

In [ ]:
#values = [i / 10 for i in range(1,20, 1)]
#res = [scipy.ndimage.gaussian_gradient_magnitude(sample[::,::,0], sigma=v) for v in values ]
#show_all(len(res) // 4, 4, *res, scale=40, titles = list(map(str,values)))

In [ ]:
logas2 = scipy.ndimage.gaussian_gradient_magnitude(data_input_normalized[::,::,::,0], 1)
print(logas2.shape)
print(scipy.stats.describe(logas2, None))

print(sample.dtype)
logas2b = scipy.ndimage.gaussian_gradient_magnitude(data_input_normalized[::,::,::,0], 1.5)
print(logas2b.shape, logas2b.dtype)
print(scipy.stats.describe(logas2b, None))

logasa = scipy.ndimage.gaussian_gradient_magnitude(data_input_normalized[2,::,::,0], 1)
print(logasa.shape)
print(scipy.stats.describe(logasa, None))

logasa2b = scipy.ndimage.gaussian_gradient_magnitude(data_input_normalized[2,::,::,0], 1.5)
print(logasa2b.shape, logasa2b.dtype)
print(scipy.stats.describe(logasa2b, None))

show_all(2,1, np.hstack([logas2[2], logas2b[2]]),
         np.hstack([logasa, logasa2b])
         , scale=20)

## Membrane

In [ ]:
from clb.dataprep.utils import extract_label_edges

def get_edges_only(labels, edge_size=6):
    boundaries = extract_label_edges(labels, edge_size)
    return labels * boundaries

show_all(1,1, np.hstack([data_input_labels[0], get_edges_only(data_input_labels[0], 6)]), scale=20)

In [ ]:
print(data_input_labels.dtype)
boundaries = get_edges_only(data_input_labels)
print(boundaries.shape, boundaries.dtype)
show_all(1,2, data_input_labels[0], boundaries[0], scale=20)

In [ ]:
from clb.image_processing import extend_membrane as get_wide_membrane

edges = get_edges_only(data_input_labels) > 0

In [ ]:
cd8_normalized = data_input_normalized[0,::,::,2]
labels_slices = data_input_labels[0]
edges_slice = edges[0]
show_all(3,1, np.hstack([cd8_normalized, get_wide_membrane(cd8_normalized, 1)]), \
          np.hstack([labels_slices * cd8_normalized, labels_slices * get_wide_membrane(cd8_normalized, 1)]), \
         np.hstack([edges_slice * cd8_normalized, edges_slice * get_wide_membrane(cd8_normalized, 1)]), scale=20)


In [ ]:
pdl1_normalized = data_input_normalized[0,::,::,1]
show_all(1,1, np.hstack([pdl1_normalized, get_wide_membrane(pdl1_normalized, 1)]), scale=20)

In [ ]:
labels_slices = data_input_labels[0]
edges_slice = edges[0]
show_all(3,1, np.hstack([pdl1_normalized, get_wide_membrane(pdl1_normalized, 1)]), \
          np.hstack([labels_slices * pdl1_normalized, labels_slices * get_wide_membrane(pdl1_normalized, 1)]), \
         np.hstack([edges_slice * pdl1_normalized, edges_slice * get_wide_membrane(pdl1_normalized, 1)]), scale=20)


### Expand outside 

In [ ]:
def get_edges_extended(labels, edge_size=6):
    boundaries = extract_label_edges(labels, edge_size)
    return labels * boundaries

In [ ]:
dilated = skimage.morphology.dilation(labels_slices, disk(4))

dilated_respect = dilated.copy()
dilated_respect[labels_slices != 0] = labels_slices[labels_slices != 0]

edges_only = get_edges_only(dilated_respect, 10)

dilated_difference = dilated_respect * (labels_slices == 0)

edges_only_small = get_edges_only(labels_slices, 4)
dilated_diff_with_edges = dilated_difference.copy()
dilated_diff_with_edges[dilated_diff_with_edges == 0] = edges_only_small[dilated_diff_with_edges == 0]

show_all(5,3, 
         pdl1_normalized, get_wide_membrane(pdl1_normalized, 1) * (get_edges_extended(labels_slices, 6) != 0), get_edges_extended(labels_slices, 6), 
         labels_slices, dilated, dilated_respect, 
         edges_only, get_wide_membrane(pdl1_normalized, 1) * edges_only, pdl1_normalized * edges_only, 
         dilated_difference, get_wide_membrane(pdl1_normalized, 1) * dilated_difference, pdl1_normalized * dilated_difference, 
         dilated_diff_with_edges, get_wide_membrane(pdl1_normalized, 1) * dilated_diff_with_edges, pdl1_normalized * dilated_diff_with_edges,
         scale=20)

### Final version

In [ ]:
import clb.classify.feature_extractor as fe

importlib.reload(clb.image_processing)
importlib.reload(fe)
print (data_input_bad_normalized.shape, data_input_bad_normalized.dtype)
pre_memb_bad, pre_memb_labels = fe.preprocess_channel(data_input_bad_normalized[::, ::,::,2], data_input_bad_labels, 'memb')
print ('volume:', pre_memb_bad.shape, pre_memb_bad.dtype)
print ('labels:', pre_memb_labels.shape, pre_memb_labels.dtype)
show_all(2,1, np.hstack([sample_bad[::,::,2], pre_memb_bad[0]]), 
                        np.hstack([data_input_bad_labels[0], pre_memb_labels[0]]), scale=30)

In [ ]:
importlib.reload(clb.image_processing)
importlib.reload(fe)
print (data_input_bad_normalized.shape, data_input_bad_normalized.dtype)
pre_memb_bad, pre_memb_labels = fe.preprocess_channel(data_input_bad_normalized[::, ::,::,2], data_input_bad_labels, 'memb')
print ('volume:', pre_memb_bad.shape, pre_memb_bad.dtype)
print ('labels:', pre_memb_labels.shape, pre_memb_labels.dtype)
show_all(2,1, np.hstack([sample_bad[::,::,2], pre_memb_bad[0]]), 
                        np.hstack([data_input_bad_labels[0], pre_memb_labels[0]]), scale=30)

## Summary

In [ ]:
import clb.classify.feature_extractor as fe

def overlay_sample_boundaries(input_sample_single, mask_image, colour=(30, 0, 0)):
    input_sample = np.stack((input_sample_single,)*3, axis=-1)
    boundary_sample = skimage.segmentation.find_boundaries(mask_image)
    input_sample[boundary_sample != 0] += colour
    input_sample[mask_image != 0] += (0, 30, 0)
    return input_sample

def summary_for_data(volume, labels, slice_num=None):
    slice_num = slice_num or len(volume) // 2
    org_vol, org_labels = volume[slice_num], labels[slice_num]
    org_overlay = overlay_sample_boundaries(org_vol, org_labels)
    
    clahe = fe.preprocess_channel(volume, labels, 'clahe')
    clahe_vol, clahe_labels = clahe[0][slice_num], clahe[1][slice_num]
    clahe_overlay = overlay_sample_boundaries(clahe_vol, clahe_labels)
    
    edges = fe.preprocess_channel(volume, labels, 'edges')
    edges_vol, edges_labels = edges[0][slice_num], edges[1][slice_num]
    edges_overlay = overlay_sample_boundaries(edges_vol, edges_labels)
    
    memb = fe.preprocess_channel(volume, labels, 'memb')
    memb_vol, memb_labels = memb[0][slice_num], memb[1][slice_num]
    memb_overlay = overlay_sample_boundaries(memb_vol, memb_labels)
    
    return show_all(3,1, np.hstack([org_vol, clahe_vol, edges_vol, memb_vol]), 
                        np.hstack([org_labels, clahe_labels, edges_labels, memb_labels]), 
                      np.hstack([org_overlay, clahe_overlay, edges_overlay, memb_overlay]), 
                    scale=30)

In [ ]:
summary_for_data(data_input_bad_normalized[::,::,::,0], data_input_bad_labels, 2)

In [ ]:
summary_for_data(data_input_bad_normalized[::,::,::,1], data_input_bad_labels, 2)

In [ ]:
summary_for_data(data_input_bad_normalized[::,::,::,2], data_input_bad_labels, 2)